In [ ]:
!pip install paddlepaddle-gpu paddleocr


Training Data Extraction

In [ ]:
import pandas as pd
import requests
import os
import io
from PIL import Image

# Load the CSV file
df = pd.read_csv('/kaggle/input/amazon-ml-school/student_resource 3/dataset/train.csv')


# Create a directory to store images
os.makedirs('downloaded_images', exist_ok=True)

# List to hold metadata
metadata_list = []

# Function to download images and store metadata
def download_images_and_store_metadata(image_link, group_id, entity_name, entity_value, index):
    try:
        # Download the image
        image_response = requests.get(image_link)
        image_response.raise_for_status()  # Raise an error for bad responses
        image_data = image_response.content
        
        # Save the image
        image_filename = os.path.join('downloaded_images', f"{group_id}_{index}.jpg")
        with open(image_filename, 'wb') as img_file:
            img_file.write(image_data)
        
        # Store metadata
        metadata_list.append({
            'group_id': group_id,
            'entity_name': entity_name,
            'entity_value': entity_value,
            'image_path': image_filename
        })
        print(f"Successfully downloaded: {image_link}")

    except Exception as e:
        print(f"Error processing image {image_link}: {e}")

# Process images and store metadata
for index, row in df.iterrows():
    image_link = row['image_link']
    group_id = row['group_id']
    entity_name = row['entity_name']
    entity_value = row['entity_value']
    download_images_and_store_metadata(image_link, group_id, entity_name, entity_value, index)

# Convert metadata list to DataFrame and save to CSV
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv('Train_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")

In [ ]:
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image

# Initialize the OCR model (use det=True for detection and rec=True for recognition)
ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu = True)  # Choose 'ch', 'en', 'fr', 'german', etc.

# Function to extract text from image
def extract_text(image_path):
    try:
        # Perform OCR using PaddleOCR
        result = ocr.ocr(image_path, cls=True)
        
        # Concatenate the recognized text segments
        extracted_text = ' '.join([line[1][0] for line in result[0]])

        return extracted_text.strip()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Apply OCR to each image and store the results in 'extracted_text'
metadata_df['extracted_text'] = metadata_df['image_path'].apply(extract_text)

# Print the updated DataFrame with extracted text
print(metadata_df)


In [ ]:
metadata_df.to_csv('Trainingdataset.csv',index = False)

Testing Data Extraction

In [ ]:
import pandas as pd
import requests
import os
import io
from PIL import Image

# Load the CSV file
df = pd.read_csv('/kaggle/input/amazon-ml-school/student_resource 3/dataset/test.csv')


# Create a directory to store images
os.makedirs('downloaded_images', exist_ok=True)

# List to hold metadata
metadata_list = []

# Function to download images and store metadata
def download_images_and_store_metadata(image_link, group_id, entity_name, index):
    try:
        # Download the image
        image_response = requests.get(image_link)
        image_response.raise_for_status()  # Raise an error for bad responses
        image_data = image_response.content
        
        # Save the image
        image_filename = os.path.join('downloaded_images', f"{group_id}_{index}.jpg")
        with open(image_filename, 'wb') as img_file:
            img_file.write(image_data)
        
        # Store metadata
        metadata_list.append({
            'group_id': group_id,
            'entity_name': entity_name,
            'image_path': image_filename
        })
        print(f"Successfully downloaded: {image_link}")

    except Exception as e:
        print(f"Error processing image {image_link}: {e}")

# Process images and store metadata
for index, row in df.iterrows():
    image_link = row['image_link']
    group_id = row['group_id']
    entity_name = row['entity_name']
    download_images_and_store_metadata(image_link, group_id, entity_name, index)

# Convert metadata list to DataFrame and save to CSV
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv('Test_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")

In [ ]:
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image

# Initialize the OCR model (use det=True for detection and rec=True for recognition)
ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu = True)  # Choose 'ch', 'en', 'fr', 'german', etc.

# Function to extract text from image
def extract_text(image_path):
    try:
        # Perform OCR using PaddleOCR
        result = ocr.ocr(image_path, cls=True)
        
        # Concatenate the recognized text segments
        extracted_text = ' '.join([line[1][0] for line in result[0]])

        return extracted_text.strip()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Apply OCR to each image and store the results in 'extracted_text'
metadata_df['extracted_text'] = metadata_df['image_path'].apply(extract_text)

# Print the updated DataFrame with extracted text
print(metadata_df)


In [ ]:
metadata_df.to_csv('Test_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")